# Learning the Linear Model

Our goal is to learn the following linear model:

\begin{gather}
X_{n+1} = T X_{n} + D
\end{gather}

When:

\begin{gather}
T =
 \begin{bmatrix} 
    A_{1} & B_{1} & C_{1} \\
    A_{2} & B_{2} & C_{2} \\
    A_{3} & B_{3} & C_{3} 
    \end{bmatrix} , 
     D =
 \begin{bmatrix} 
    D_{1}\\
    D_{2}\\
    D_{3} 
    \end{bmatrix} , 
       X_{n} =
 \begin{bmatrix} 
    HR_{n}\\
    BP_{n}\\
    RR_{n} 
    \end{bmatrix}
\end{gather}

# Model Training
With given 3 signals, blood pressure heart rate and respiration rate, from second 0 to n, lets assume the following equations:

\begin{gather}
 \begin{bmatrix} HR_{1} \\ \vdots \\ HR_{n} \end{bmatrix}=
 \begin{bmatrix} 
    HR_{0} & BP_{0} & RR_{0} \\
    \vdots & \vdots & \vdots \\
    HR_{n-1} & BP_{n-1} & RR_{n-1} 
    \end{bmatrix}
  \begin{bmatrix}
   A_{1}   \\
   B_{1}  \\
   C_{1} 
   \end{bmatrix} +
   \begin{bmatrix} D_{1} \\ \vdots \\ D_{1} \end{bmatrix}
\end{gather}

\begin{gather}
 \begin{bmatrix} BP_{1} \\ \vdots \\ BP_{n} \end{bmatrix}=
 \begin{bmatrix} 
    HR_{0} & BP_{0} & RR_{0} \\
    \vdots & \vdots & \vdots \\
    HR_{n-1} & BP_{n-1} & RR_{n-1} 
    \end{bmatrix}
  \begin{bmatrix}
   A_{2}   \\
   B_{2}  \\
   C_{2} 
   \end{bmatrix} +
   \begin{bmatrix} D_{2} \\ \vdots \\ D_{2} \end{bmatrix}
\end{gather}

\begin{gather}
 \begin{bmatrix} RR_{1} \\ \vdots \\ RR_{n} \end{bmatrix}=
 \begin{bmatrix} 
    HR_{0} & BP_{0} & RR_{0} \\
    \vdots & \vdots & \vdots \\
    HR_{n-1} & BP_{n-1} & RR_{n-1} 
    \end{bmatrix}
  \begin{bmatrix}
   A_{3}   \\
   B_{3}  \\
   C_{3} 
   \end{bmatrix} +
   \begin{bmatrix} D_{3} \\ \vdots \\ D_{3} \end{bmatrix}
\end{gather}

Each one of the equations above describing a linear model, accordingly we will try to regress the following for i= {1,2,3}:

\begin{gather}
  \begin{bmatrix}
   A_{i}   \\
   B_{i}  \\
   C_{i} 
   \end{bmatrix} , D_{i}
\end{gather}

Methods for regression:
1. Least Squares (Manually while assuming D=0), for example:

\begin{gather}
    \overline{
    \begin{bmatrix}
        A_{1}\\B_{1}\\C_{1}
    \end{bmatrix}}= 
    \left(
        \begin{bmatrix}
            HR_{0}&BP_{0}&RR_{0}\\ \vdots&\vdots&\vdots\\ HR_{n-1} & BP_{n-1} & RR_{n-1}
        \end{bmatrix}^T
        \begin{bmatrix}
            HR_{0}&BP_{0}&RR_{0}\\ \vdots&\vdots&\vdots\\ HR_{n-1} & BP_{n-1} & RR_{n-1}
        \end{bmatrix}
    \right)^{-1}
    \begin{bmatrix}
        HR_{0}&BP_{0}&RR_{0}\\ \vdots&\vdots&\vdots\\ HR_{n-1} & BP_{n-1} & RR_{n-1}
    \end{bmatrix}^T
    \begin{bmatrix}
        HR_{1}\\ \vdots\\ HR_{n}
    \end{bmatrix}
\end{gather}

2. Least Squares (sklearn)

3. LassoCV (sklearn)

4. RidgeCV (sklearn)

manual page for the methods form sklearn library:
https://scikit-learn.org/stable/modules/linear_model.html

In [ ]:
Model = namedtuple('Model', ['T', 'D', 'start_train_point', 'end_train_point', 'patient_id', 'reg_method', 'scaler'])
def estimate_T(start_train_point, end_train_point, patient_id, reg_method):

    D = np.array([[0],[0],[0]])
    
    reg1=reg2=reg3=None
    scaler=None
    
    time, bp_train = bp(start_train_point,end_train_point,patient_id,0,0)
    time, hr_train = hr(start_train_point,end_train_point,patient_id,0,0)
    time, rr_train = rr(start_train_point,end_train_point,patient_id,0,0)
    
    # scaling
    data_train = np.column_stack((hr_train, bp_train, rr_train))
    scaler = StandardScaler()
    scaler.fit(data_train)
    data_train_scaled = scaler.transform(data_train)
    bp_train = data_train_scaled[:,1]
    hr_train = data_train_scaled[:,0]
    rr_train = data_train_scaled[:,2]
    
    # x is the 3 signals data from 0 to n-1
    x = np.column_stack((hr_train, bp_train, rr_train))
    x = np.delete(x, -1, axis=0)

    # y_i is the data from 1 to n for the i signal
    y_bp = np.delete(bp_train, 0)
    y_rr = np.delete(rr_train, 0)
    y_hr = np.delete(hr_train, 0)
    
    if reg_method == 1:
        hr_coef = np.dot(np.linalg.inv(np.dot(x.transpose(),x)),np.dot(x.transpose(),y_hr))
        bp_coef = np.dot(np.linalg.inv(np.dot(x.transpose(),x)),np.dot(x.transpose(),y_bp))
        rr_coef = np.dot(np.linalg.inv(np.dot(x.transpose(),x)),np.dot(x.transpose(),y_rr))
    
    else:
        if reg_method == 2:
            reg1 = linear_model.LinearRegression(fit_intercept=False)
            reg2 = linear_model.LinearRegression(fit_intercept=False)
            reg3 = linear_model.LinearRegression(fit_intercept=False)
            
        if reg_method == 3:
            reg1 = linear_model.LassoCV(cv = 5, fit_intercept=False)
            reg2 = linear_model.LassoCV(cv = 5, fit_intercept=False)
            reg3 = linear_model.LassoCV(cv = 5, fit_intercept=False)

        if reg_method == 4:
            reg1 = linear_model.RidgeCV(fit_intercept=False)
            reg2 = linear_model.RidgeCV(fit_intercept=False)
            reg3 = linear_model.RidgeCV(fit_intercept=False)
        
        reg1.fit(x, y_hr)
        hr_coef = reg1.coef_
        hr_intercept = reg1.intercept_
        
        reg2.fit(x, y_bp)
        bp_coef = reg2.coef_
        bp_intercept = reg2.intercept_
        
        reg3.fit(x, y_rr)
        rr_coef = reg3.coef_
        rr_intercept = reg3.intercept_
        
        D = np.row_stack((hr_intercept, bp_intercept, rr_intercept))
        
    D = scaler.mean_.reshape(3,1)
    T = np.row_stack((hr_coef, bp_coef, rr_coef))
    model = Model(T, D, start_train_point, end_train_point, patient_id, reg_method, scaler)
    
    return model

# Model Testing

Lets test the regressed matrix by caculating the following:

\begin{gather}
\overline{X}_{n+1} = \overline{T} X_{n} + \overline{D}
\end{gather}


In [ ]:
Prediction = namedtuple('Prediction', ['time', 'data_test', 'data_pred', 'patient_id', 'model', 'white_noise_rank'])
def test_T(model, start_point, end_point, patient_id):

    time, bp_test = bp(start_point,end_point,patient_id,0,0)
    time, hr_test = hr(start_point,end_point,patient_id,0,0)
    time, rr_test = rr(start_point,end_point,patient_id,0,0)
    
    # scaling
    data_test = np.column_stack((hr_test, bp_test, rr_test))
    scaler = StandardScaler()
    scaler.fit(data_test)
    data_test_scaled = scaler.transform(data_test)
    
    data_pred = np.dot(model.T, data_test_scaled.transpose())# + np.repeat(model.D, len(time), axis=1)
    
    # reverse scaling
    data_pred = model.scaler.inverse_transform(data_pred.transpose())
    data_pred = data_pred.transpose()

    time = np.delete(time, 0)
    data_test = np.delete(data_test.transpose(), 0, axis=1)
    data_pred = np.delete(data_pred, -1, axis=1)

    prediction = Prediction(time, data_test, data_pred, patient_id, model, white_noise(data_test, data_pred))
    
    return prediction

# White Noise testing

In [ ]:
def white_noise(data_test, data_pred):
    
    error = data_test - data_pred
    left_rank_mean = right_rank_mean = rank_mean = 0
    rank_ac = 0 # autocorelation
    
    for signal in [0,1,2]:
        for epsilon, bonous in [(0.2,1), (0.4,10), (0.6,100), (0.8,1000), (1,10000), (1.4,100000), (1.8,1000000), (2.2,10000000)]:
            if np.abs(np.mean(error[signal])) < epsilon:
                rank_mean = rank_mean + bonous
            if np.abs(np.mean(error[signal][0:(int)(len(error[signal])/2)])) < epsilon:
                left_rank_mean = left_rank_mean + bonous
            if np.abs(np.mean(error[signal][(int)(len(error[signal])/2):])) < epsilon:
                right_rank_mean = right_rank_mean + bonous
                
                
        for alpha_, bonous in [(0.15,1), (0.10,10), (0.08,100), (0.05,1000), (0.03,10000), (0.02,100000), (0.01,1000000)]:
            h, pV, Q, cV = lbqtest(error[signal], range(1, min(20, len(error[signal]))), alpha=alpha_)
            if not any(h):      
                rank_ac = rank_ac + bonous
    return (rank_mean, rank_ac, left_rank_mean, right_rank_mean)

# Printing Prediction Results

In [ ]:
def print_prediction_results(predictions):
    if(type(predictions) != list):
        predictions = [[predictions]]
    
    start = np.inf
    end = -np.inf
    for prediction_one_model in predictions:
        for prediction in prediction_one_model:
            start = min(start, prediction.time[0])
            end = max(end, prediction.time[-1])
    time_total, bp_original_total = bp(start,end,predictions[0][0].patient_id,0,0)
    time_total, hr_original_total = hr(start,end,predictions[0][0].patient_id,0,0)
    time_total, rr_original_total = rr(start,end,predictions[0][0].patient_id,0,0)
    
    error_color = 'gray'
    original_color = 'black'
    prediction_colors = ['C0', 'C1', 'C2', 'C3', 'C4']
    
    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(24, 16))
    
    ax[0].set_title('Heart Rate')
    ax[0].set_xlabel('Time [sec]')
    ax[0].set_ylabel('Heart Rate [bpm]')
    ax[0].plot(time_total, hr_original_total, linewidth=2, color=original_color, label='Data')    
    ax2_hr = ax[0].twinx()
    ax2_hr.set_ylabel('Error [bpm]', color=error_color)
    ax2_hr.tick_params(axis='y', labelcolor=error_color)

    ax[1].set_title('Systolic Blood Pressure')
    ax[1].set_xlabel('Time [sec]')
    ax[1].set_ylabel('Systolic Blood Pressure [mmHg]')
    ax[1].plot(time_total, bp_original_total, linewidth=2, color=original_color, label='Data')
    ax2_bp = ax[1].twinx()
    ax2_bp.set_ylabel('Error [mmHg]', color=error_color)
    ax2_bp.tick_params(axis='y', labelcolor=error_color)


    ax[2].set_title('Respiration Rate')
    ax[2].set_xlabel('Time [sec]')
    ax[2].set_ylabel('Respiration Rate [Hz]')
    ax[2].plot(time_total, rr_original_total, linewidth=2, color=original_color, label='Data')
    ax2_rr = ax[2].twinx()
    ax2_rr.set_ylabel('Error [Hz]', color=error_color)
    ax2_rr.tick_params(axis='y', labelcolor=error_color)
    
    model_index = 0
    for prediction_one_model in predictions:
        model_index = model_index + 1
        print("*******************************************************************************************************")
        print("Model Number: ", model_index)
        print("Estimation Method: ", prediction.model.reg_method)
        print("T=\n", prediction.model.T)
        print("D=\n", prediction.model.D)
        print("Training Period: [%s, %s]" % (prediction.model.start_train_point, prediction.model.end_train_point))
        print("Training based on %s data" % (prediction.model.patient_id))
        for prediction in prediction_one_model:
            print("Testing the model based on %s data on the period [%s, %s]"  % (prediction.patient_id, prediction.time[0], prediction.time[-1]))
            print('Mean squared error: %.2f'
              % mean_squared_error(prediction.data_test, prediction.data_pred))
            # The coefficient of determination: 1 is perfect prediction
            print('Coefficient of determination: %.2f'
              % r2_score(prediction.data_test, prediction.data_pred))
            print("The Noise Is White: ", white_noise(prediction.data_test, prediction.data_pred))

            ax[0].plot(prediction.time, prediction.data_pred[0], linewidth=2, color=prediction_colors[model_index-1], label="Predicted by model " + str(model_index))
            ax2_hr.plot(prediction.time, prediction.data_test[0] - prediction.data_pred[0], '--', color=error_color, linewidth=0.5)
            
            ax[1].plot(prediction.time, prediction.data_pred[1], linewidth=2, color=prediction_colors[model_index-1], label="Predicted by model " + str(model_index))
            ax2_bp.plot(prediction.time, prediction.data_test[1] - prediction.data_pred[1], ':', color=error_color, linewidth=0.5)
            
            ax[2].plot(prediction.time, prediction.data_pred[2], linewidth=2, color=prediction_colors[model_index-1], label="Predicted by model " + str(model_index))
            ax2_rr.plot(prediction.time, prediction.data_test[2] - prediction.data_pred[2], '--', color=error_color, linewidth=0.5)
        print("*******************************************************************************************************")
            
    ax[0].legend(loc='best')
    ax[1].legend(loc='best')
    ax[2].legend(loc='best')
    plt.show()